In [1]:
import pandas as pd
import requests
import certifi

# Dallas coordinates
latitude  = 32.7767
longitude = -96.7970

start_date = "2023-01-01"
end_date   = "2023-12-31"

# Replace precipitation with pressure + wind direction
hourly_vars = [
    "temperature_2m",
    "surface_pressure",
    "windspeed_10m",
    "winddirection_10m",
    "relative_humidity_2m"
]

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date":   end_date,
    "hourly":     ",".join(hourly_vars),
    "temperature_unit": "celsius",
    "windspeed_unit": "kmh",
    "pressure_unit": "hPa",
    "timezone": "auto"
}

# 👇 THIS is the important line
ca_path = certifi.where()

response = requests.get(url, params=params, timeout=120, verify=ca_path)
response.raise_for_status()

data = response.json()

df = pd.DataFrame(data['hourly'])
df['time'] = pd.to_datetime(df['time'])

# Add metadata
df['Latitude']  = latitude
df['Longitude'] = longitude
df['Location']  = "Dallas"

print(df.head())

                 time  temperature_2m  surface_pressure  windspeed_10m  \
0 2023-01-01 00:00:00            16.3             994.2           19.0   
1 2023-01-01 01:00:00            15.5             994.1           17.9   
2 2023-01-01 02:00:00            15.0             994.2           17.4   
3 2023-01-01 03:00:00            14.8             994.3           18.9   
4 2023-01-01 04:00:00            14.6             994.4           17.4   

   winddirection_10m  relative_humidity_2m  Latitude  Longitude Location  
0                207                    58   32.7767    -96.797   Dallas  
1                208                    60   32.7767    -96.797   Dallas  
2                202                    61   32.7767    -96.797   Dallas  
3                198                    61   32.7767    -96.797   Dallas  
4                202                    62   32.7767    -96.797   Dallas  


In [2]:
# Make a copy to work with
real_dallas = df.copy()

# Split datetime into date/time components
real_dallas['Year']  = real_dallas['time'].dt.year
real_dallas['Month'] = real_dallas['time'].dt.month
real_dallas['Day']   = real_dallas['time'].dt.day
real_dallas['Time']  = real_dallas['time'].dt.time

# Map to clearer column names
real_dallas['Temperature_C']     = real_dallas['temperature_2m']
real_dallas['Humidity_pct']      = real_dallas['relative_humidity_2m']
real_dallas['Wind_Speed_kmh']    = real_dallas['windspeed_10m']
real_dallas['Pressure_hPa']      = real_dallas['surface_pressure']
real_dallas['Wind_Direction_deg'] = real_dallas['winddirection_10m']

# Keep only the columns you care about in a nice order
real_dallas_aligned = real_dallas[[
    'Location',
    'Temperature_C',
    'Humidity_pct',
    'Wind_Speed_kmh',
    'Pressure_hPa',
    'Wind_Direction_deg',
    'Time',
    'Year',
    'Month',
    'Day'
]]

print(real_dallas_aligned.head())
print(real_dallas_aligned.shape)

  Location  Temperature_C  Humidity_pct  Wind_Speed_kmh  Pressure_hPa  \
0   Dallas           16.3            58            19.0         994.2   
1   Dallas           15.5            60            17.9         994.1   
2   Dallas           15.0            61            17.4         994.2   
3   Dallas           14.8            61            18.9         994.3   
4   Dallas           14.6            62            17.4         994.4   

   Wind_Direction_deg      Time  Year  Month  Day  
0                 207  00:00:00  2023      1    1  
1                 208  01:00:00  2023      1    1  
2                 202  02:00:00  2023      1    1  
3                 198  03:00:00  2023      1    1  
4                 202  04:00:00  2023      1    1  
(8760, 10)


In [3]:
real_dallas_aligned["id"] = range(1, len(real_dallas_aligned) + 1)

C:\Users\batur\AppData\Local\Temp\ipykernel_15624\3363209308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_dallas_aligned["id"] = range(1, len(real_dallas_aligned) + 1)


In [4]:
real_dallas_aligned

,Location,Temperature_C,Humidity_pct,Wind_Speed_kmh,Pressure_hPa,Wind_Direction_deg,Time,Year,Month,Day,id
0,Dallas,16.3,58,19.0,994.2,207,00:00:00,2023,1,1,1
1,Dallas,15.5,60,17.9,994.1,208,01:00:00,2023,1,1,2
2,Dallas,15.0,61,17.4,994.2,202,02:00:00,2023,1,1,3
3,Dallas,14.8,61,18.9,994.3,198,03:00:00,2023,1,1,4
4,Dallas,14.6,62,17.4,994.4,202,04:00:00,2023,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
8755,Dallas,11.5,58,16.2,1003.1,13,19:00:00,2023,12,31,8756
8756,Dallas,10.3,63,19.4,1003.7,18,20:00:00,2023,12,31,8757
8757,Dallas,9.0,63,19.4,1004.4,18,21:00:00,2023,12,31,8758
8758,Dallas,7.6,61,19.4,1005.3,15,22:00:00,2023,12,31,8759


In [5]:
#out_path = r"C:\PythonGIS\dataanalysis\proj\real\weather_Dallas_2023.csv"

#import os
#os.makedirs(os.path.dirname(out_path), exist_ok=True)

#real_dallas_aligned.to_csv(out_path, index=False)
#print("Saved:", out_path)

In [6]:
# Toronto coordinates
latitude  = 43.6532
longitude = -79.3832

start_date = "2023-01-01"
end_date   = "2023-12-31"

# Replace precipitation with pressure + wind direction
hourly_vars = [
    "temperature_2m",
    "surface_pressure",
    "windspeed_10m",
    "winddirection_10m",
    "relative_humidity_2m"
]

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date":   end_date,
    "hourly":     ",".join(hourly_vars),
    "temperature_unit": "celsius",
    "windspeed_unit": "kmh",
    "pressure_unit": "hPa",
    "timezone": "auto"
}

# 👇 THIS is the important line
ca_path = certifi.where()

response = requests.get(url, params=params, timeout=120, verify=ca_path)
response.raise_for_status()

data = response.json()

df = pd.DataFrame(data['hourly'])
df['time'] = pd.to_datetime(df['time'])

# Add metadata
df['Latitude']  = latitude
df['Longitude'] = longitude
df['Location']  = "Toronto"

print(df.head())

                 time  temperature_2m  surface_pressure  windspeed_10m  \
0 2023-01-01 00:00:00             3.0             999.6           11.8   
1 2023-01-01 01:00:00             2.7            1000.0            8.4   
2 2023-01-01 02:00:00             2.6            1000.4            9.8   
3 2023-01-01 03:00:00             2.6            1001.2           11.0   
4 2023-01-01 04:00:00             2.4            1001.1            8.8   

   winddirection_10m  relative_humidity_2m  Latitude  Longitude Location  
0                293                    91   43.6532   -79.3832  Toronto  
1                277                    92   43.6532   -79.3832  Toronto  
2                264                    92   43.6532   -79.3832  Toronto  
3                259                    89   43.6532   -79.3832  Toronto  
4                258                    89   43.6532   -79.3832  Toronto  


In [7]:
# Make a copy to work with
real_toronto = df.copy()

# Split datetime into date/time components
real_toronto['Year']  = real_toronto['time'].dt.year
real_toronto['Month'] = real_toronto['time'].dt.month
real_toronto['Day']   = real_toronto['time'].dt.day
real_toronto['Time']  = real_toronto['time'].dt.time

# Map to clearer column names
real_toronto['Temperature_C']     = real_toronto['temperature_2m']
real_toronto['Humidity_pct']      = real_toronto['relative_humidity_2m']
real_toronto['Wind_Speed_kmh']    = real_toronto['windspeed_10m']
real_toronto['Pressure_hPa']      = real_toronto['surface_pressure']
real_toronto['Wind_Direction_deg'] = real_toronto['winddirection_10m']

# Keep only the columns you care about in a nice order
real_toronto_aligned = real_toronto[[
    'Location',
    'Temperature_C',
    'Humidity_pct',
    'Wind_Speed_kmh',
    'Pressure_hPa',
    'Wind_Direction_deg',
    'Time',
    'Year',
    'Month',
    'Day'
]]

print(real_toronto_aligned.head())
print(real_toronto_aligned.shape)

  Location  Temperature_C  Humidity_pct  Wind_Speed_kmh  Pressure_hPa  \
0  Toronto            3.0            91            11.8         999.6   
1  Toronto            2.7            92             8.4        1000.0   
2  Toronto            2.6            92             9.8        1000.4   
3  Toronto            2.6            89            11.0        1001.2   
4  Toronto            2.4            89             8.8        1001.1   

   Wind_Direction_deg      Time  Year  Month  Day  
0                 293  00:00:00  2023      1    1  
1                 277  01:00:00  2023      1    1  
2                 264  02:00:00  2023      1    1  
3                 259  03:00:00  2023      1    1  
4                 258  04:00:00  2023      1    1  
(8760, 10)


In [8]:
real_toronto_aligned["id"] = range(1, len(real_toronto_aligned) + 1)

C:\Users\batur\AppData\Local\Temp\ipykernel_15624\3754554218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_toronto_aligned["id"] = range(1, len(real_toronto_aligned) + 1)


In [9]:
real_toronto_aligned

,Location,Temperature_C,Humidity_pct,Wind_Speed_kmh,Pressure_hPa,Wind_Direction_deg,Time,Year,Month,Day,id
0,Toronto,3.0,91,11.8,999.6,293,00:00:00,2023,1,1,1
1,Toronto,2.7,92,8.4,1000.0,277,01:00:00,2023,1,1,2
2,Toronto,2.6,92,9.8,1000.4,264,02:00:00,2023,1,1,3
3,Toronto,2.6,89,11.0,1001.2,259,03:00:00,2023,1,1,4
4,Toronto,2.4,89,8.8,1001.1,258,04:00:00,2023,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
8755,Toronto,0.8,87,12.6,1003.6,53,19:00:00,2023,12,31,8756
8756,Toronto,0.9,92,14.0,1003.7,67,20:00:00,2023,12,31,8757
8757,Toronto,0.8,92,13.7,1003.9,63,21:00:00,2023,12,31,8758
8758,Toronto,0.7,91,14.1,1003.9,58,22:00:00,2023,12,31,8759


In [10]:
#out_path = r"C:\PythonGIS\dataanalysis\proj\real\weather_Toronto_2023.csv"

#import os
#os.makedirs(os.path.dirname(out_path), exist_ok=True)

#real_toronto_aligned.to_csv(out_path, index=False)
#print("Saved:", out_path)

In [11]:
# Montreal coordinates
latitude  = 45.5017
longitude = -73.5673

start_date = "2023-01-01"
end_date   = "2023-12-31"

# Replace precipitation with pressure + wind direction
hourly_vars = [
    "temperature_2m",
    "surface_pressure",
    "windspeed_10m",
    "winddirection_10m",
    "relative_humidity_2m"
]

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date":   end_date,
    "hourly":     ",".join(hourly_vars),
    "temperature_unit": "celsius",
    "windspeed_unit": "kmh",
    "pressure_unit": "hPa",
    "timezone": "auto"
}

# 👇 THIS is the important line
ca_path = certifi.where()

response = requests.get(url, params=params, timeout=120, verify=ca_path)
response.raise_for_status()

data = response.json()

df = pd.DataFrame(data['hourly'])
df['time'] = pd.to_datetime(df['time'])

# Add metadata
df['Latitude']  = latitude
df['Longitude'] = longitude
df['Location']  = "Montreal"

print(df.head())

                 time  temperature_2m  surface_pressure  windspeed_10m  \
0 2023-01-01 00:00:00             2.5             994.4           10.8   
1 2023-01-01 01:00:00             2.6             994.2           11.2   
2 2023-01-01 02:00:00             2.5             994.6            8.8   
3 2023-01-01 03:00:00             2.4             995.1            7.6   
4 2023-01-01 04:00:00             2.4             995.4            9.9   

   winddirection_10m  relative_humidity_2m  Latitude  Longitude  Location  
0                360                   100   45.5017   -73.5673  Montreal  
1                  6                   100   45.5017   -73.5673  Montreal  
2                348                   100   45.5017   -73.5673  Montreal  
3                295                    99   45.5017   -73.5673  Montreal  
4                280                    99   45.5017   -73.5673  Montreal  


In [12]:
# Make a copy to work with
real_montreal = df.copy()

# Split datetime into date/time components
real_montreal['Year']  = real_montreal['time'].dt.year
real_montreal['Month'] = real_montreal['time'].dt.month
real_montreal['Day']   = real_montreal['time'].dt.day
real_montreal['Time']  = real_montreal['time'].dt.time

# Map to clearer column names
real_montreal['Temperature_C']     = real_montreal['temperature_2m']
real_montreal['Humidity_pct']      = real_montreal['relative_humidity_2m']
real_montreal['Wind_Speed_kmh']    = real_montreal['windspeed_10m']
real_montreal['Pressure_hPa']      = real_montreal['surface_pressure']
real_montreal['Wind_Direction_deg'] = real_montreal['winddirection_10m']

# Keep only the columns you care about in a nice order
real_montreal_aligned = real_montreal[[
    'Location',
    'Temperature_C',
    'Humidity_pct',
    'Wind_Speed_kmh',
    'Pressure_hPa',
    'Wind_Direction_deg',
    'Time',
    'Year',
    'Month',
    'Day'
]]

print(real_montreal_aligned.head())
print(real_montreal_aligned.shape)

   Location  Temperature_C  Humidity_pct  Wind_Speed_kmh  Pressure_hPa  \
0  Montreal            2.5           100            10.8         994.4   
1  Montreal            2.6           100            11.2         994.2   
2  Montreal            2.5           100             8.8         994.6   
3  Montreal            2.4            99             7.6         995.1   
4  Montreal            2.4            99             9.9         995.4   

   Wind_Direction_deg      Time  Year  Month  Day  
0                 360  00:00:00  2023      1    1  
1                   6  01:00:00  2023      1    1  
2                 348  02:00:00  2023      1    1  
3                 295  03:00:00  2023      1    1  
4                 280  04:00:00  2023      1    1  
(8760, 10)


In [13]:
real_montreal_aligned["id"] = range(1, len(real_montreal_aligned) + 1)

C:\Users\batur\AppData\Local\Temp\ipykernel_15624\4143945243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_montreal_aligned["id"] = range(1, len(real_montreal_aligned) + 1)


In [14]:
real_montreal_aligned

,Location,Temperature_C,Humidity_pct,Wind_Speed_kmh,Pressure_hPa,Wind_Direction_deg,Time,Year,Month,Day,id
0,Montreal,2.5,100,10.8,994.4,360,00:00:00,2023,1,1,1
1,Montreal,2.6,100,11.2,994.2,6,01:00:00,2023,1,1,2
2,Montreal,2.5,100,8.8,994.6,348,02:00:00,2023,1,1,3
3,Montreal,2.4,99,7.6,995.1,295,03:00:00,2023,1,1,4
4,Montreal,2.4,99,9.9,995.4,280,04:00:00,2023,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
8755,Montreal,-4.3,64,9.7,1007.4,266,19:00:00,2023,12,31,8756
8756,Montreal,-5.3,67,5.2,1007.8,326,20:00:00,2023,12,31,8757
8757,Montreal,-5.7,62,12.1,1007.8,12,21:00:00,2023,12,31,8758
8758,Montreal,-6.5,60,11.8,1007.8,20,22:00:00,2023,12,31,8759


In [15]:
#out_path = r"C:\PythonGIS\dataanalysis\proj\real\weather_Montreal_2023.csv"

#import os
#os.makedirs(os.path.dirname(out_path), exist_ok=True)

#real_montreal_aligned.to_csv(out_path, index=False)
#print("Saved:", out_path)